This notebook is a supplement to the paper "Negative events - Logical and philosophical aspects"[link] by Lucas Champollion[link] and Timothée Bernard[link].

The notebook uses the [Natural Language Toolkit](https://www.nltk.org)'s interface to [Prover9/Mace4](https://www.cs.unm.edu/~mccune/prover9/) to investigate the derivability of conclusions from the logical principles in the paper, understood as algebraic equations.

## Outline

In [127]:
from nltk.test.inference_fixt import setup_module

setup_module()

from nltk import *
#from nltk.sem.drt import DrtParser

from nltk.sem.logic import *
logic._counter._value = 0

from nltk.sem import Expression
read_expr = Expression.fromstring

## Axioms

### Lattice axioms

In [128]:
or_id = read_expr('Or(x,x) = x')
and_id = read_expr('And(x,x) = x')

or_comm = read_expr('Or(x,y) = Or(y,x)')
and_comm = read_expr('And(x,y) = And(y,x)')

or_assoc = read_expr('Or(x,Or(y,z)) = Or(Or(x,y),z)')
and_assoc = read_expr('And(x,And(y,z)) = And(And(x,y),z)')

or_absorp = read_expr('Or(x,And(x,y)) = x')
and_absorp = read_expr('And(x,Or(x,y)) = x')

lattice = [or_id, and_id, or_comm, and_comm, or_assoc, and_assoc, or_absorp, and_absorp]

### Bounded lattice axioms

In [129]:
bot = read_expr('Or(x, Bot) = x')
top = read_expr('And(x, Top) = x')

bounded_lattice = lattice + [bot, top]

### Distributive lattice axioms

In [130]:
dist = read_expr('And(x,Or(y,z)) = Or(And(x,y),And(x,z))')

dist_bounded_lattice = bounded_lattice + [dist]

### Complete lattice axioms

In [158]:
uniq_union = read_expr('all P.(exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)))')
uniq_join = read_expr('all P.(exists z.(all x.(P(x) -> And(x, z) = z) & all y.((all x.(P(x) -> And(x, y) = y)) -> And(y,z) = y)))')

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 86034 was started by guzzler on oahzzdepbm,
Sun Jun 19 10:14:57 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 21.
% Level of proof is 9.
% Maximum clause weight is 36.000.
% Given clauses 21.


1 (all P exists x ((all y (P(y) -> And(y,x) = y)) & (all z ((all y (P(y) -> And(y,z) = y)) -> And(z,x) = x)))).  [assumption].
2 (all P exists x ((all y (P(y) -> And(y,x) = y)) & (all z ((all y (P(y) -> And(y,z) = y)) -> And(z,x) = x)))).  [goal].
3 -P(x) | And(x,f1(y)) = x.  [clausify(1)].
4 P(f2(x,y)) | And(y,f1(x)) = f1(x).  [clausify(1)].
5 And(f2(x,y)

### Boolean algebra axioms

In [159]:
compl = read_expr('all x.(exists y. (And(x, y) = Bot & Or(x, y) = Top))')

boolean_algebra = dist_bounded_lattice + [uniq_union, uniq_join, compl]

### Exclusion axioms

In [160]:
excl_cumm = read_expr('Excl(x1, y1) & Excl(x2, y2) -> Excl(And(x1, x2), And(y1, y2))')
excl_symm = read_expr('Excl(x, y) <-> Excl(y, x)')

### Possibility axioms

In [161]:
#define conflict:
confl_def = read_expr('confl(x, y) <-> exists z1.(And(z1, x) = x & exists z2.(And(z2, y) = y & Excl(z1, z2)))')

#define possibilities:
int_poss_def = read_expr('int_poss(x) <-> -Excl(x, x)')
ext_poss_def = read_expr('ext_poss(x) <-> exists y.(int_poss(y) & -exists z.(And(y, z) = z & int_poss(z) & -(confl(z,x))))')

#define world:
world_def = read_expr('world(x) <-> (int_poss(x) & all y.((And(x, y) = y) -> -int_poss(y)))')

#define actual:
actual_def = read_expr('actual(x) <-> world(actual_world) & And(x, actual_world) = x')

#Harmony
poss_harm = read_expr('all x.(-int_poss(x) -> -ext_poss(x))')

#Nirvana
poss_nirv = read_expr('all x.(-confl(Bot, x))')

#Rashōmon
poss_rshm = read_expr('all x y.((int_poss(x) & int_poss(y) & -confl(x, y)) -> int_poss(And(x,y)))')

#Cosmopolitanism
poss_cosm = read_expr('all x.(int_poss(x) -> exists y.(world(y) & And(x, y) = y))')

event_space = boolean_algebra + [excl_cumm, excl_symm, confl_def, int_poss_def, ext_poss_def, world_def, actual_def, poss_harm, poss_nirv, poss_rshm, poss_cosm]

## Theorems

### Plenitude

In [185]:
#check plenitude
goal = read_expr('all x.(int_poss(x) -> ext_poss(x))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 86859 was started by guzzler on oahzzdepbm,
Sun Jun 19 10:43:42 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 12.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 40.


9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
14 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
15 (all x (int_poss(x) -> ext_poss(x))).  [goal].
24 -world(x) | int_poss(x).  [clausify(9)].
25 -world(x) | And(x,y) != y | -int_poss(y).  [clausify(9)].
27 -int_poss(x) | world(f11(x)).  [cla

### Distributivity of actuality

In [186]:
#check distributivity of actuality
goal = read_expr('all x.((exists y. actual(y) & And(x, y) = x) -> actual(x))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 86861 was started by guzzler on oahzzdepbm,
Sun Jun 19 10:43:44 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 11.
% Level of proof is 4.
% Maximum clause weight is 9.000.
% Given clauses 58.


9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
10 actual(x) <-> world(actual_world) & And(x,actual_world) = x.  [assumption].
15 (all x ((exists y actual(y)) & And(x,y) = x -> actual(x))).  [goal].
27 And(x,x) = x.  [assumption].
56 -world(x) | int_poss(x).  [clausify(9)].
57 -world(x) | And(x,y) != y | -int_poss(y).  [c

### Cumulativity of actuality

In [187]:
#check cumulativity of actuality
goal = read_expr('all P.((all x.(P(x) -> actual(x))) -> (exists z.(all x.(P(x) -> And(x, z) = x) & all y.((all x.(P(x) -> And(x, y) = x)) -> And(y,z) = z)) & actual(z)))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 86864 was started by guzzler on oahzzdepbm,
Sun Jun 19 10:43:46 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 6.
% Maximum clause weight is 16.000.
% Given clauses 61.


6 confl(x,y) <-> (exists z (And(z,x) = x & (exists u (And(u,y) = y & Excl(z,u))))).  [assumption].
7 int_poss(x) <-> -Excl(x,x).  [assumption].
9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
12 (all x -confl(Bot,x)).  [assumption].
14 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
23 And(x

### Manicaeism

In [168]:
#check Manichaeism
goal = read_expr('all x.((world(x) -> all y.(And(x, y) = x | confl(x, y))))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 86289 was started by guzzler on oahzzdepbm,
Sun Jun 19 10:23:17 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 13.
% Level of proof is 4.
% Maximum clause weight is 10.000.
% Given clauses 44.


9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
14 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
15 (all x (world(x) -> (all y (And(x,y) = x | confl(x,y))))).  [goal].
30 -world(x) | int_poss(x).  [clausify(9)].
31 -world(x) | And(x,y) != y | -int_poss(y).  [clausify(9)].
33 -int_poss(x) |

### Classicality of conjunction

In [195]:
#check classicality of conjunction
goal = read_expr(r'((exists x. (P(x) & actual(x))) & (exists y. (Q(y) & actual(y)))) <-> exists y exists z.(actual(y) & actual(z) & P(y) & Q(z) & actual(Or(y, z)))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 87726 was started by guzzler on oahzzdepbm,
Sun Jun 19 11:20:59 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 6.
% Maximum clause weight is 16.000.
% Given clauses 78.


6 confl(x,y) <-> (exists z (And(z,x) = x & (exists u (And(u,y) = y & Excl(z,u))))).  [assumption].
7 int_poss(x) <-> -Excl(x,x).  [assumption].
9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
12 (all x -confl(Bot,x)).  [assumption].
14 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
23 And(x

### Classicality of disjunction

In [198]:
#check classicality of disjunction
goal = read_expr(r'(exists x. (actual(x) & (P(x) | Q(x)))) <-> ((exists x. (actual(x) & P(x))) & (exists y. (actual(y) & Q(y))))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 92467 was started by guzzler on oahzzdepbm,
Sun Jun 19 14:32:05 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 6.
% Maximum clause weight is 16.000.
% Given clauses 66.


6 confl(x,y) <-> (exists z (And(z,x) = x & (exists u (And(u,y) = y & Excl(z,u))))).  [assumption].
7 int_poss(x) <-> -Excl(x,x).  [assumption].
9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
12 (all x -confl(Bot,x)).  [assumption].
14 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
23 And(x

### No Gaps

In [209]:
#define negating a set
def negate(x, P):
    return '(all a. (P(a) -> (exists b. And(x,b) = b & Excl(a, b))) & (all c. ((And(c, x) = c & (-exists d. (And(x, d) = d & And(c, d) = d))) -> (exists d. (P(d) & Excl(d, c))))) & all e all f. ((e != f) -> (exists g exists h.(And(g, x) = g & And(h, x) = h & g!=h & Excl(e, g) & Excl(f, h)))))'

#check no gaps
goal = read_expr('(-exists x. (actual(x) & P(x))) <-> (exists y. (actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 94354 was started by guzzler on oahzzdepbm,
Sun Jun 19 22:20:06 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.00) seconds.
% Length of proof is 17.
% Level of proof is 6.
% Maximum clause weight is 16.000.
% Given clauses 126.


6 confl(x,y) <-> (exists z (And(z,x) = x & (exists u (And(u,y) = y & Excl(z,u))))).  [assumption].
7 int_poss(x) <-> -Excl(x,x).  [assumption].
9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
12 (all x -confl(Bot,x)).  [assumption].
14 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
23 And(

### No Gluts

In [210]:
#check no gluts
goal = read_expr('-((exists x. actual(x) & P(x)) & (exists y. actual(y) &'+negate('y', 'P')+'))')
prover = Prover9Command(goal, assumptions = event_space)
prover.prove()
print(prover.proof())

============================== prooftrans ============================
Prover9 (64) version 2009-11A, November 2009.
Process 94360 was started by guzzler on oahzzdepbm,
Sun Jun 19 22:20:15 2022
The command was "/usr/local/bin/prover9".
============================== end of head ===========================

============================== end of input ==========================

============================== PROOF =================================

% -------- Comments from original proof --------
% Proof 1 at 0.01 (+ 0.01) seconds.
% Length of proof is 16.
% Level of proof is 5.
% Maximum clause weight is 10.000.
% Given clauses 67.


9 world(x) <-> int_poss(x) & (all y (And(x,y) = y -> -int_poss(y))).  [assumption].
10 actual(x) <-> world(actual_world) & And(x,actual_world) = x.  [assumption].
14 (all x (int_poss(x) -> (exists y (world(y) & And(x,y) = y)))).  [assumption].
15 -((exists x actual(x)) & P(x) & (exists y actual(y)) & (all a (P(a) -> (exists b And(x,b) = b) & Excl(a,b))) & 